In [ ]:
# import libraries
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as T
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# need to change your output.zip folder dir get from training
!unzip -q /content/drive/MyDrive/VisualPollution/outputs.zip -d '/content/'

In [ ]:
# hyper parameters
device = 'cuda'
IMG_SIZE = (256, 256)
NUM_CLASS = 11

In [ ]:
# load model
def load_model(model_path):
    model = torchvision.models.efficientnet_b0()

    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features=in_features, out_features=NUM_CLASS)
    model = model.to(device)

    model.load_state_dict(torch.load(model_path))

    return model

model_path = '/content/content/outputs/checkpoint.pt'
model = load_model(model_path)
model.eval();
# print(model)

In [ ]:
# class dictionary
idx2class = {0: 'Bad Billboard',
 1: 'Bad Streetlight',
 2: 'Broken Signage',
 3: 'Clutter Sidewalk',
 4: 'Construction Road',
 5: 'Faded Signage',
 6: 'Garbage',
 7: 'Graffiti',
 8: 'Potholes',
 9: 'Sand On Road',
 10: 'Unkept Facade'}

In [ ]:
# image transformation
Transform = T.Compose(
                    [T.ToTensor(),
                    T.Resize(IMG_SIZE),
                    # T.RandomRotation(90),
                    # T.RandomHorizontalFlip(p=0.5),
                    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

# predict image
def predict(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    orig_image = image.copy()

    image = Transform(image) 
    image = torch.unsqueeze(image, 0)
    image = image.to(device)

    # Forward pass throught the image.
    outputs = model(image)
    outputs = outputs.cpu().detach().numpy()
    # print(outputs)

    pred_class_name = idx2class[np.argmax(outputs[0])]

    return orig_image,  pred_class_name

# show image
def show_image(image, pred):
    plt.imshow(image)
    plt.title(pred)
    plt.axis('off')
    plt.show()

In [ ]:
# prediction on single image
img_path = "/content/drive/MyDrive/VisualPollution/test_image/6985665000_c3bd140d9d_n.jpg"
image, pred = predict(img_path)
show_image(image, pred)


In [ ]:
# need to change your test_image.zip dir
!unzip -q /content/drive/MyDrive/test_image.zip -d '/content/'

In [ ]:
# prediction on image folder
test_dir = "/content/drive/MyDrive/VisualPollution/test_image/"
for img in glob.glob(test_dir + "*"):
    image, pred = predict(img)
    show_image(image, pred)